# 10. 주어진 환경과 상호작용하며 학습하는 DQN

## 10.0. 요약

### 10.0.1 지난장

 지난 9장에서는 2014년 등장하여 신경망의 미래가 될 것으로 불리는 GAN에 대해서 알아보았다. GAN은 생성자와 분류자로 구성되며, 이 둘간의 경쟁적인 관계를 통해 지속적으로 학습하여 결과를 예측하는 신경망이다. 예제를 통해 기초적인 GAN 신경망을 구축했으며, 특정 Label을 동시에 제공함으로써 생성자가 Label에 따라서 결과값을 생성하는 cGAN(Conditional GAN)에 대해서 살펴보았다.

### 10.0.1 이번장

 이번 장에는 ML 학습 방법 중 마지막 방법으로 불리는 '강화학습' 방법의 DQN(Deep Queue Network)에 대해서 살펴보려고 한다. 강화학습은 기존의 지도/비지도 학습과 달리 학습을 통한 보상을 통해 실시간으로 학습이 수행되는 방식이다. 이는 2013년 Atari 게임들을 통해 능력이 검증되었으며, 2016년 알파고의 알고리즘에도 사용되기도 했다. 그래서 이번 장에는 강화학습에 대한 전반적인 이야기와 Cart-Pole이라는 게임에 강화학습을 적용해보는 예제를 살펴보도록 한다.
 
## 10.1. 강화학습과 DQN 기초

강화학습은 주어진 환경과 상호작용하여 좋은 점수를 얻는 방향으로 성장하는 머신러닝 분야이다. 강화학습 모델은 주어진 환경에서 시행착오를 겪으며 좋은 피드백을 받는 쪽으로 최적화하며 성장하게 된다.

### 강화학습의 요소
1. 에이전트: 인공지능 플레이어
2. 환경: 에이전트가 솔루션을 찾기 위한 무대
3. 행동: 에이전트가 환경 안에서 시행하는 상호작용
4. 보상: 에이전트의 행동에 따른 점수 혹은 결과

![basic_structure](img/basic_structure.png)

## 10.2. 카트폴 게임 마스터하기

게임을 마스터하는 것은 최고의 점수를 받는 것을 의미합니다. 그래서 위의 게임에서 강화학습을 활용할 때, 얻게 되는 점수가 보상이됩니다. 

DQN의 주요 특징은 크게 두가지로 기억하기(memorize)와 다시 보기(replay)입니다. 

In [1]:
import gym
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import matplotlib.pyplot as plt

1. EPISODE: 총 플레이할 게임의 수
2. 엡실론(EPS_START, EPS_END): 에이전트가 무작위로 행동할 확률. 예를들어 엡실론이 50%라면 절반의 확률로 무작위로 행동하고, 절반의 확률로는 학습된 방향으로 행동한다. 무작위로 행동하는 이유는 에이전트가 가능한 모든 행동을 경험하도록 하기 위함이다. 시작값은 EPS_START인 90%이며, 학습이 진행되며 조금씩 감소하여 EPS_END인 5%까지 내려가게 됩니다.
3. 엡실론 감소율(EPS_DECAY): EPS_START로 시작하는 엡실론의 값을 EPS_END까지 점진적으로 감소시키는 값이다.
4. 감마(GAMMA): 감마는 에이저트가 현재 보상을 미래 보상보다 얼마나 가치있게 여기는지에 대한 값이다. 예를 들어 1년 뒤에 받을 만원가 지금의 만원의 가치는 같지 않아, 1년 뒤의 가치를 이자율 만큼 할인해야 한다. 다른 말로는 할인계수라고도 한다.

In [2]:
EPISODES = 50
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
GAMMA = 0.8
LR = 0.001
BATCH_SIZE = 64

### 10.2.3. 이전 경험 기억하기(memorize)

DQN이 등장하기 이전의 신경망을 강화학습에 적용하는데 아래와 같은 문제점이 있었다.
1. 딥러닝 모델들의 일반적인 학습 데이터 샘플은 각각 독립적이라고 가정하는데, 강화학습에서는 연속된 상태가 강한 상관관계가 있으므로 학습이 어렵다는 점이다. 무작위로 가져오지 않고 연속적인 경험을 학습할 때 초반의 몇 가지 경험 패턴에만 치중해서 학습하게 되어 최적의 행동 패턴을 찾기가 어렵다
2. 신경망이 새로운 경험을 전 경험에 겹쳐쓰며 쉽게 잊어버린다.

위의 두 가지 문제를 해결하기 위해 등장한 것이 기억하기(memorize)기능이다. 이전 경험들을 계속 배열에 담아 재학습시킴으로써 신경망이 잊지 않도록 하는것이다. 기억한 경험들은 학습시에 무작위로 뽑아 경험 간의 상관관계를 줄인다.

경헙은 각각의 상태, 행동, 보상 등을 담아야 한다.

### 10.2.4. 행동하기(act)

무직위로 숫자를 골라 엡실론 값보다 높으면 신경망이 학습하여 행동하고, 낮으면 무작위로 행동하게 함수를 구성한다. 엡실론 값이 큰 학습 초반에는 에이전트가 하는 행동에 의미를 부여할 수 없으므로 다양한 경험을 유도하고, 점점 엡실론 값이 낮아 질 수록 신경망이 결정하는 비율을 높인다. 이를 엡실론 그리디(엡실론 탐욕) 알고리즘이라고 한다.

### 10.2.5. 경험으로부터 배우기

경험들이 모이면, 이 경험들을 반복적으로 학습해야 한다. DQN에서도 이런 학습과정을 replay라고 부른다. 이 역할을 learn()함수가 수행하게 되고, 신경망의 기억(self.memory)에 쌓인 경험을 토대로 학습을 진행한다.

self.memory에 저장된 경험들의 수가 배치 크기보다 작을 떄는 return으로 학습을 거그로, 충분히 쌓이게 되면 self.memory 큐에서 무작위로 배치크기 만큼의 경험을 가져오게된다. 경험을 무작위로 들고옴으로써 경험간에 발생하는 상관관계를 줄일 수 있게 된다.

In [3]:
class DQNAgent:
    def __init__(self):
        self.model = nn.Sequential(
            nn.Linear(4, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )
        self.optimizer = optim.Adam(self.model.parameters(), LR)
        self.steps_done = 0
        self.memory = deque(maxlen=10000)

    def memorize(self, state, action, reward, next_state):
        self.memory.append((state,
                            action,
                            torch.FloatTensor([reward]),
                            torch.FloatTensor([next_state])))
    
    def act(self, state):
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1
        if random.random() > eps_threshold:
            return self.model(state).data.max(1)[1].view(1, 1)
        else:
            return torch.LongTensor([[random.randrange(2)]])
    
    def learn(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        states, actions, rewards, next_states = zip(*batch)

        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)

        '''
        에이전트가 점진적으로 발전된 행동을 하도록, 신경망이 주어진 상태에서 각 행동의 가치를 더 잘 예측하도록 학습해야 한다.
        따라서, 현재 상태를 신경망에 통과시켜 왼쪽 / 오른쪽으로 가는 행동에 대한 가치를 계산한다. [via gather() 함수]
        따라서 '현 상태에서 했던 행동의 가치'들을 current_q에 담는다.
        '''
        
        current_q = self.model(states).gather(1, actions)
        
        '''
        DQN 알고리즘의 학습은 할인된 미래가치로 누적된 보상을 극대화 하는 방향으로 이루어진다.
        미래가치: 에이전트가 미래에 받을 수 있는 보상의 기대값.
        max() 함수를 통해 다음상태(next_state)에서 에이전트가 생각하는 행동의 최대가치를 구해 max_next_q에 담는다.
        
        그리고, 할인계수를 통해 현재의 가치를 미래의 행동으로 얻는 가치에 비해 높게 책정한다.
        '''
        
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q)
        
        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()



In [4]:
env = gym.make('CartPole-v0')
agent = DQNAgent()
score_history = []

for e in range(1, EPISODES+1):
    state = env.reset()
    steps = 0
    while True:
        env.render()
        state = torch.FloatTensor([state])
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action.item())

        # 게임이 끝났을 경우 마이너스 보상주기
        if done:
            reward = -1

        agent.memorize(state, action, reward, next_state)
        agent.learn()

        state = next_state
        steps += 1

        if done:
            print("에피소드:{0} 점수: {1}".format(e, steps))
            score_history.append(steps)
            break

에피소드:1 점수: 24
에피소드:2 점수: 16
에피소드:3 점수: 14
에피소드:4 점수: 14
에피소드:5 점수: 18
에피소드:6 점수: 16
에피소드:7 점수: 11
에피소드:8 점수: 11
에피소드:9 점수: 13
에피소드:10 점수: 11
에피소드:11 점수: 10
에피소드:12 점수: 15
에피소드:13 점수: 12
에피소드:14 점수: 21
에피소드:15 점수: 14
에피소드:16 점수: 11
에피소드:17 점수: 10
에피소드:18 점수: 12
에피소드:19 점수: 14
에피소드:20 점수: 13
에피소드:21 점수: 12
에피소드:22 점수: 11
에피소드:23 점수: 25
에피소드:24 점수: 17
에피소드:25 점수: 33
에피소드:26 점수: 71
에피소드:27 점수: 16
에피소드:28 점수: 21
에피소드:29 점수: 45
에피소드:30 점수: 153
에피소드:31 점수: 83
에피소드:32 점수: 50
에피소드:33 점수: 170
에피소드:34 점수: 200
에피소드:35 점수: 183
에피소드:36 점수: 190
에피소드:37 점수: 158
에피소드:38 점수: 188
에피소드:39 점수: 186
에피소드:40 점수: 165
에피소드:41 점수: 166
에피소드:42 점수: 163
에피소드:43 점수: 198
에피소드:44 점수: 185
에피소드:45 점수: 178
에피소드:46 점수: 188
에피소드:47 점수: 186
에피소드:48 점수: 200
에피소드:49 점수: 190
에피소드:50 점수: 186
